In [390]:
import random

# StringIO behaves like a file object 
from io import StringIO 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

import copy


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

import joblib

%matplotlib inline 

In [391]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, low, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.low = list(low)
        self.up = list(up)

In [392]:
def generate_cqk_problem(n):
    d = []
    low = []
    up = []
    b = []
    a = []
    temp = 0
    lb = 0.0
    ub = 0.0
    lower = 10
    upper = 25
    r = 0

    for i in range(n):
        
        b.append(10 + 14*random.random())
        low.append(1 + 14*random.random())
        up.append(1 + 14*random.random())
        if low[i] > up[i]:
            temp = low[i]
            low[i] = up[i]
            up[i] = temp
        
        lb = lb + b[i]*low[i];
        ub = ub + b[i]*up[i];
        
        #Uncorrelated
        d.append(random.randint(10,25))
        a.append(random.randint(10,25))
        
    r = lb + (ub - lb)*0.7;
    
    return CqkProblem( r, n, d, a, b, low, up)

In [393]:

def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [394]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [395]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton(p):
    lambs = [] 
    derivs = []
    phis = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    
    it = 1
    
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        sum_slopes = sum(slopes)
        
    if phi == 0.0:
        return it,lambs,derivs,phis,sum_slopes
    elif alfa == beta:
        return -1,lambs,derivs,phis,sum_slopes
    else:
        return -2,lambs,derivs,phis,sum_slopes

In [396]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def new_newton(p,slopes):
    lambs = [] 
    derivs = []
    phis = []
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    x = []
    r = p.r
    
    lamb = 8.88138793613069
    deriv = 26525.250920050636
    phi = -100.90696513877992
    
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)

    it = 1
    
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        sum_slopes = sum(slopes)
        
    if phi == 0.0:
        return it,lambs,derivs,phis,sum_slopes
    elif alfa == beta:
        return -1,lambs,derivs,phis,sum_slopes
    else:
        return -2,lambs,derivs,phis,sum_slopes

In [362]:

c = ''
with open("instance_test_1000x20000_edited_init_lamb.txt", "r") as fd:
    c = StringIO(fd.read())
    
d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 

feature_names = ['a','b','d','r','inicital_lamb','init_phi','init_deriv','sum_slopes','second_lambda','third_lamb','fourth_lamb','second_phi','second_deriv','final_lamb']




In [363]:
knapsack = {"data":d, "feature_names": feature_names}
dataset = pd.DataFrame(knapsack['data'], columns = knapsack['feature_names'])

In [369]:
# Coletando x e y

X_lamb_init = dataset.iloc[:,:-9]
y_lamb_init = dataset['second_lambda'].values


In [367]:
del dataset['inicital_lamb']

In [385]:
scaler=joblib.load('scaler_initial_lamb.bin')
loaded_model = joblib.load('finalized_model_initial_lamb.sav')
list_init_lamb = []
for index, row in X_lamb_init.iterrows():
    x_values_lamb_init = scaler.transform(np.asarray([row]))
    list_init_lamb.append(loaded_model.predict(x_values_lamb_init)[0])

In [388]:
dataset.insert(4, "initial_lamb", list_init_lamb, True)

In [389]:
dataset

,a,b,d,r,initial_lamb,init_phi,init_deriv,sum_slopes,second_lambda,third_lamb,fourth_lamb,second_phi,second_deriv,final_lamb
0,17.563,17.137339,17.040,153.789637,6.842696,-10980.003494,6519.650825,19633.199608,8.545164,8.699830,8.702720,-819.579639,5299.040793,8.702723
1,17.693,16.860005,17.378,152.173745,7.154606,-10733.790555,6209.600581,18705.260923,8.874408,9.058557,9.062409,-939.632669,5102.561985,9.062409
2,17.427,17.091528,17.568,154.809150,7.206331,-12648.973495,7183.312077,19052.168439,8.922230,9.252729,9.263391,-1803.698408,5457.502063,9.263391
3,17.586,17.111238,17.412,155.648969,7.147207,-11665.257903,7289.355120,19248.063742,8.715292,8.917252,8.922372,-1172.620970,5806.202783,8.922369
4,17.582,17.216332,17.283,154.783176,6.948409,-10668.697650,6876.022142,19428.736025,8.545581,8.788623,8.794218,-1321.161360,5435.941733,8.794218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19989,17.411,17.016685,17.556,152.326709,7.134725,-11114.647829,7196.731251,18856.980521,8.655831,8.911568,8.912931,-1367.302879,5346.505145,8.912931
19990,17.353,16.815114,17.340,151.900359,7.176845,-10269.059269,7059.747514,18550.279684,8.660109,8.825953,8.829031,-971.013709,5854.988059,8.829031
19991,17.500,16.873401,17.507,152.081341,7.214224,-10496.290822,7170.427011,18524.373004,8.690782,8.927505,8.931318,-1337.699584,5650.902346,8.931323
19992,17.582,17.116986,17.458,149.919828,6.862662,-12707.758254,7378.410411,19187.950882,8.562879,8.892406,8.909531,-1769.309063,5369.248407,8.909558


In [308]:
X_train_lamb_init, X_test_lamb_init, y_train_lamb_init, y_test_lamb_init = train_test_split(X_lamb_init, y_lamb_init)

In [309]:
# Padronização
scaler = StandardScaler()
scaler.fit(X_train_lamb_init)

# Aplicando a padronização aos dados
X_train_p = scaler.transform(X_train_lamb_init)
X_test_p = scaler.transform(X_test_lamb_init)

In [315]:
joblib.dump(scaler, 'scaler_second_lamb.bin', compress=True)

['scaler_second_lamb.bin']

In [316]:
# mlp = MLPClassifier(hidden_layer_sizes = (30,30,30))
regr_lamb_second = MLPRegressor(activation='logistic',random_state=1, max_iter=10000).fit(X_train_p, y_train_lamb_init)

In [333]:
regr_lamb_second.fit(X_train_p, y_train_lamb_init)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [334]:
finalized_model = 'finalized_model_second_lamb.sav'
joblib.dump(regr_lamb_second, finalized_model)

['finalized_model_second_lamb.sav']

In [335]:
model = joblib.load('finalized_model_second_lamb.sav') 
model.score(X_test_p, y_test_lamb_init)

0.9967608303295321

In [319]:
regr_lamb_init.predict(X_test_p[:10])

array([8.53990286, 8.63960156, 8.64482309, 8.67526504, 8.59986872,
       8.70223379, 8.71827128, 8.62117701, 8.76436515, 8.57692405])

In [323]:
y_test_lamb_init[:10]

array([8.540671, 8.635876, 8.647532, 8.673911, 8.599964, 8.702917,
       8.71377 , 8.622853, 8.762779, 8.574168])

In [124]:
# Coletando x e y

X_lamb_final = dataset.iloc[:,:-4]
y_lamb_final = dataset['final_lamb'].values

X_lamb_final

,a,b,d,r,inicital_lamb,init_phi,init_deriv,sum_slopes
0,17.4940,17.029666,17.5942,152.885619,7.161592,-54687.558901,35106.292801,93999.049662
1,17.4562,16.978139,17.5174,152.332254,7.145984,-54264.307048,35022.668258,93817.521347
2,17.5278,16.992513,17.4684,152.854383,7.133927,-54269.736971,34861.587275,94248.309884
3,17.5012,17.034433,17.4804,152.194909,7.046928,-54017.708919,34860.245225,94866.566281
4,17.4994,16.924226,17.5592,151.461236,7.139020,-53643.567519,34297.676209,93311.216624
...,...,...,...,...,...,...,...,...
9988,17.5362,17.029213,17.5334,151.224695,7.036200,-52336.987119,35305.604760,94416.033391
9989,17.6030,17.039789,17.5520,152.282310,7.080079,-55609.928752,34310.654507,94504.400377
9990,17.4690,17.107762,17.6414,151.084173,7.023715,-55532.227707,34416.401995,94541.681598
9991,17.5060,17.094586,17.5394,151.827955,7.028284,-54231.369650,35286.391565,94923.053496


In [125]:
X_train_lamb_final, X_test_lamb_final, y_train_lamb_final, y_test_lamb_final = train_test_split(X_lamb_final, y_lamb_final)

In [126]:
# Padronização
scaler_final = StandardScaler()
scaler_final.fit(X_train_lamb_final)

# Aplicando a padronização aos dados
X_train_p_final = scaler_final.transform(X_train_lamb_final)
X_test_p_final = scaler_final.transform(X_test_lamb_final)

In [127]:
# mlp = MLPClassifier(hidden_layer_sizes = (30,30,30))
regr_lamb_final = MLPRegressor(activation='logistic',random_state=1, max_iter=10000).fit(X_train_p_final, y_train_lamb_final)

In [128]:
regr_lamb_final.fit(X_train_p_final, y_train_lamb_final)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [131]:
regr_lamb_final.score(X_test_p_final, y_test_lamb_final)

0.9465677502698139

In [139]:
n = 5000
p = generate_cqk_problem(n)

In [279]:
%%time

it,lambs, derivs,phis,sum_slopes = newton(p)

it, lambs, derivs, phis

CPU times: user 36.7 ms, sys: 2.28 ms, total: 38.9 ms
Wall time: 37.6 ms


(5,
 [7.109283902249071,
  8.68877391425142,
  8.887313754588348,
  8.890987896434018,
  8.890989911913914],
 [33505.547648770495,
  26525.250920050636,
  25558.88397434641,
  25531.41635363121,
  25531.41635363121],
 [-52921.677857901836,
  -5266.3190825638,
  -93.90696513877992,
  -0.051458056391680884,
  1.6692069948476274e-10])

In [213]:
def phi_lambda_predicted(p,lamb,r):
    deriv = 0.0
    phi = r * -1
    x = []
    sum_slope = 0
    slopes = []
    for i in range(p.n):
        slope = (p.b[i]/p.d[i])*p.b[i]
        sum_slope = sum_slope + slope
        slopes.append(slope)
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slope
        phi = phi + p.b[i] * x[i];
    return deriv, phi,sum_slope, slopes

In [274]:
%%time

som_a = 0
som_b = 0
som_d = 0

for i in range(p.n):
    som_a = som_a + p.a[i]
    som_b = som_b + p.b[i]
    som_d = som_d + p.d[i]

new_a = som_a/p.n
new_b = som_b/p.n
new_d = som_d/p.n
rn = p.r/p.n

x_values_lamb_init = scaler.transform(np.asarray([[new_a, new_b, new_d,rn ]]))
first_lamb = regr_lamb_init.predict(x_values_lamb_init)[0]
# first_lamb = 7.109283902249071

CPU times: user 4.01 ms, sys: 142 µs, total: 4.15 ms
Wall time: 4.1 ms


In [276]:
%%time
first_deriv, first_phi, sum_slope,slopes = phi_lambda_predicted(p,first_lamb,p.r)

x_values_lamb_final = scaler_final.transform(np.asarray([[new_a, new_b, new_d,rn,first_lamb,first_phi, first_deriv, sum_slope]]))

predicted_lamb = regr_lamb_final.predict(x_values_lamb_final)[0]


CPU times: user 18.6 ms, sys: 1.15 ms, total: 19.7 ms
Wall time: 19.6 ms


In [277]:
%%time

new_newton(p,slopes)

CPU times: user 23.8 ms, sys: 1.25 ms, total: 25.1 ms
Wall time: 27.3 ms


(3,
 [8.890978398843256, 8.890989911913945],
 [25531.41635363121, 25531.41635363121],
 [-0.2939450012458451, 5.994138518872205e-10],
 93523.62732712529)